In [33]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [34]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from wandb.integration.keras import WandbMetricsLogger

import wandb
import params

In [ ]:
def train(config):
    run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="training")

In [ ]:
def get_data():
    processed_data_at = run.use_artifact(f'{params.PROCESSED_DATA_AT}:latest')
    processed_dataset_dir = Path(processed_data_at.download())
    cwd = Path(os.getcwd())
    path = processed_dataset_dir.relative_to(cwd)
    
    return f'{path}/flowers'



wandb: Downloading large artifact flowers_simples_split:latest, 941.12MB. 8631 files... 
wandb:   8631 of 8631 files downloaded.  
Done. 0:0:21.0


In [38]:
def read_data():
    df = pd.read_csv("data_split.csv")

    dataset_dir = get_data()

    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    # Gerador para treinamento
    train_generator = datagen.flow_from_directory(
        directory=dataset_dir,
        target_size=(180, 180),
        batch_size=32,
        class_mode='categorical',
        subset='training' 
    )

    # Gerador para validação
    validation_generator = datagen.flow_from_directory(
        directory=dataset_dir,
        target_size=(180, 180),
        batch_size=32,
        class_mode='categorical',
        subset='validation'

    return df, train_generator, validation_generator
)

,fname,tulip,dandelion,daisy,rose,sunflower
0,8723767533_9145dec4bd_n.jpg,1,0,0,0,0
1,5716293002_a8be6a6dd3_n.jpg,1,0,0,0,0
2,8521597402_4b6169ba05.jpg,1,0,0,0,0
3,8713388322_e5ae26263b_n.jpg,1,0,0,0,0
4,4624404489_11e10fcd33_n.jpg,1,0,0,0,0


In [39]:
df, train_generator, validation_generator = read_data()


Found 3457 images belonging to 5 classes.
Found 860 images belonging to 5 classes.


In [40]:
def create_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(32, activation='relu'),
        layers.Dense(5, activation='softmax') 
    ])

    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    model.summary()
    return model

model = create_model()

/home/gabs/miniconda3/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 178, 178, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 89, 89, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 253472)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │     8,111,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,112,197 (30.95 MB)

 Trainable params: 8,112,197 (30.95 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
wandb.watch(model, log="all")

In [41]:
def train_model():
    history = model.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator,
        callbacks=[wandb.keras.WandbMetricsLogger(),
                        wandb.keras.WandbModelCheckpoint(filepath='model.keras', save_best_only=True)])

/home/gabs/miniconda3/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 44s 389ms/step - accuracy: 0.2460 - loss: 2.8902 - val_accuracy: 0.2326 - val_loss: 1.6010
Epoch 2/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 42s 386ms/step - accuracy: 0.2430 - loss: 1.5759 - val_accuracy: 0.3291 - val_loss: 1.5081
Epoch 3/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 44s 401ms/step - accuracy: 0.3465 - loss: 1.4799 - val_accuracy: 0.3791 - val_loss: 1.4577
Epoch 4/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 44s 401ms/step - accuracy: 0.3701 - loss: 1.3998 - val_accuracy: 0.3988 - val_loss: 1.4230
Epoch 5/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 45s 417ms/step - accuracy: 0.3956 - loss: 1.3408 - val_accuracy: 0.4000 - val_loss: 1.4388
Epoch 6/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 42s 388ms/step - accuracy: 0.4241 - loss: 1.2364 - val_accuracy: 0.4291 - val_loss: 1.3509
Epoch 7/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 45s 408ms/step - accuracy: 0.4877 - loss: 1.0998 - val_accuracy: 0.4860 - val_loss: 1.2684
Epoch 8/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 41s 376ms/step - accuracy: 0.6228 - loss: 0

In [42]:
wandb.finish()

epoch/accuracy,▁▁▂▃▃▄▅▆▇█
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▆▅▅▄▃▃▂▁
epoch/val_accuracy,▁▃▄▅▅▆▇▇▇█
epoch/val_loss,█▆▅▅▅▃▂▁▅▃
epoch/accuracy,0.74226
epoch/epoch,9
epoch/learning_rate,0.001
epoch/loss,0.68896
epoch/val_accuracy,0.53605
